# How to do multistart optimizations

Sometimes you want to make sure that your optimization is robust to the initial
parameter values, i.e. that it does not get stuck at a local optimum. This is where
multistart comes in handy.


## What does multistart (not) do

In short, multistart iteratively runs local optimizations from different initial
conditions. If enough local optimization convergence to the same point, it stops.
Importantly, it cannot guarantee that the result is the global optimum, but it can
increase your confidence in the result.

## TL;DR

To activate multistart at the default options, pass `multistart=True` to the `minimize`
or `maximize` function, as well as finite bounds on the parameters (which are used to
sample the initial points). The default options are discussed below.

In [ ]:
import numpy as np
import optimagic as om


def fun(x):
    return x @ x


x0 = np.arange(7) - 4

res = om.minimize(
    fun=fun,
    x0=x0,
    algorithm="scipy_neldermead",
    bounds=om.Bounds(
        lower=np.full_like(x0, -3),
        upper=np.full_like(x0, 8),
    ),
    multistart=True,
)

## What does multistart mean in optimagic?

Our multistart optimizations are inspired by the [TikTak algorithm](https://github.com/serdarozkan/TikTak) and consist of the following steps:

1. Draw a large exploration sample of parameter vectors randomly or using a
   low-discrepancy sequence.
1. Evaluate the objective function in parallel on the exploration sample.
1. Sort the parameter vectors from best to worst according to their objective function
   values. 
1. Run local optimizations iteratively. That is, the first local optimization is started
   from the best parameter vector in the sample. All subsequent ones are started from a
   convex combination of the currently best known parameter vector and the next sample
   point. 

## Visualizing multistart results

To illustrate the multistart results, we will consider the optimization of a slightly
more complex objective function, compared to the `fun` above.

In [ ]:
def alpine(x):
    return np.sum(np.abs(x * np.sin(x) + 0.1 * x))


res = om.minimize(
    alpine,
    x0=x0,
    algorithm="scipy_neldermead",
    bounds=om.Bounds(lower=np.full_like(x0, -3), upper=np.full_like(x0, 8)),
    multistart=om.MultistartOptions(n_samples=200, convergence_max_discoveries=3),
)

om.criterion_plot(res, max_evaluations=1_000)

In the above image we see the optimization history for all of the local optimizations
that have been run by multistart. The turquoise line represents the history
corresponding to the local optimization that found the overall best parameter.

We see that running a single optimization would not have sufficed, as some local
optimizations are stuck.

## Multistart does not always run many optimization

Since the local optimizations are run iteratively by multistart, it is possible that
only a handful of optimizations are actually run, if all of them converge to the same
point. This convergence is determined by the `convergence_max_discoveries` parameter,
which defaults to 2. This means that if 2 local optimizations report the same point,
multistart will stop. Below we see that if we use the simpler objective function
(`fun`), with `convergence_max_discoveries` set to 3, multistart runs 3 local
optimizations, and then stops, as all of them converge to the same point.

In [ ]:
res = om.minimize(
    fun,
    x0=x0,
    algorithm="scipy_neldermead",
    bounds=om.Bounds(lower=np.full_like(x0, -3), upper=np.full_like(x0, 8)),
    multistart=om.MultistartOptions(n_samples=200, convergence_max_discoveries=3),
)

om.criterion_plot(res, max_evaluations=1_000)

## How to configure multistart?

Configuration of multistart can be done by passing an instance of
`optimagic.MultistartOptions` to `maximize` or `minimize`. Let's look at an extreme
example where we manually set everything to it's default value:

1. How to run a specific number of optimizations?
1. How to set convergence options
   - These are not the convergence criteria of the local optimizations
1. How to configure the explorative sampling?
    - Know you have good start values -> sample close to those
    - No knowledge -> uniform
    - Custom sample
1. Parallelization
    - n_cores
    - batch_size
    - batch_evaluator
1. Randomness
    - seed

## Understanding multistart results

- Mention that the result is the same as the local optim result object of the best
local optimization. Additional info is only found in the multistart_info attribute.


When activating multistart, the optimization result object has the additional attribute
`multistart_info`. It is a dictionary with the following keys:
    
- `local_optima`: A list with the results from all local optimizations that were performed.
- `start_parameters`: A list with the start parameters from those optimizations 
- `exploration_sample`: A list with parameter vectors at which the objective function was evaluated in an initial exploration phase. 
- `exploration_results`: The corresponding objective values.

### Start parameters

The start parameters are the parameter vectors from which the local optimizations were
started. Since the default number of `convergence_max_discoveries` is 2, and both
local optimizations were successfull, the start parameters have 2 rows.

In [ ]:
res.multistart_info["start_parameters"]

### Local Optima

The local optima are the results from the local optimizations. Since in this example
only two local optimizations were run, the local optima list has two elements, each of
which is an optimization result object.

In [ ]:
len(res.multistart_info["local_optima"])

### Exploration sample

The exploration sample is a list of parameter vectors at which the objective function
was evaluated. Since the parameter dimension is 5, and the default number of samples is
100 times the number of parameters, the exploration sample has 500 elements.

In [ ]:
np.row_stack(res.multistart_info["exploration_sample"]).shape

### Exploration results

The exploration results are the objective function values at the exploration sample.

In [ ]:
len(res.multistart_info["exploration_results"])